In [1]:
import os
import copy
import random
import gym
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

from itertools import permutations
from sklearn.model_selection import KFold, GridSearchCV

from multiprocessing import set_start_method
import multiprocessing as mp

path = os.path.abspath('..')
if path not in sys.path:
    sys.path.append(path)

from seal.agents.default_config import DEFAULT_CONFIG as config
# from seal.agents.dqn import DQNAgent
# from seal.agents.qr_dqn import QuantileAgent
from seal.agents.multi_head_dqn import MultiHeadDQNAgent
# from seal.agents.discrete_bcq import DiscreteBCQAgent

from seal.algos.kfold import CVS, KFoldCV
from seal.algos.advantage_learner import AdvantageLearner
from seal.algos.behavior_cloning import BehaviorCloning
from seal.algos.density_ratio import VisitationRatioModel
from seal.algos.fqe import FQE

def one_step(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
    path = './data/mh/rem/trajs_mh.pkl'
    nfolds = 5
    n_splits = 5
    ckpts = (np.arange(10) + 1)*5000
    
    num_actions = 5
    # configures
    config['online'] = False
    config['lr'] = 5e-4
    config['decay_steps'] = 50000
    config['max_training_steps'] = 50000
    config['training_steps_to_checkpoint'] = 5000
    config['training_steps_to_eval'] = 100000
    config['hiddens'] = [64,64]
    config['double'] = False
    config['dueling'] = False
    config['num_heads'] = 200

    index = pd.MultiIndex.from_product([np.arange(nfolds), ckpts])
    columns = ['dqn',  'seal']
    rets = pd.DataFrame(index=index, columns=columns)

    print('-'*20, 'start', '-'*20)
    cvs = CVS(path, n_splits=nfolds, random_state=seed)
    cvs.split()
    for fold in range(nfolds):
        train_path = cvs.train_paths[fold] + 'trajs.pkl'
        kf = KFoldCV(train_path, n_trajs=None, n_splits=n_splits, shuffle=False, random_state=seed)
        kf.split()

        print('-'*20, 'training agent', '-'*20)
        # agent
        config['persistent_directory'] = kf.agent_path
        config['checkpoint_path'] = kf.ckpt_path
        agent = MultiHeadDQNAgent(num_actions=num_actions, config=config)
        agent.learn()

        print('-'*20, 'training agents', '-'*20)
        # agent_1, ..., agent_K
        for idx in range(kf.n_splits):
            config_idx = copy.deepcopy(config)
            config_idx['persistent_directory'] = kf.agent_paths[idx]
            config_idx['checkpoint_path'] = kf.ckpt_paths[idx]
            agent_idx = MultiHeadDQNAgent(num_actions=num_actions, config=config_idx)
            agent_idx.learn()

        # fitted q evaluation
        test_path = cvs.test_paths[fold] + 'trajs.pkl'
        with open(test_path, 'rb') as f:
            trajs = pickle.load(f)

        print('-'*20, 'behavior cloning', '-'*20)
        # behavior cloning
        bc = BehaviorCloning(num_actions=num_actions)
        states  = np.array([transition[0] for traj in kf.trajs for transition in traj])
        actions = np.array([transition[1] for traj in kf.trajs for transition in traj])
        bc.train(states, actions)

        for ckpt in ckpts:
            print('-'*20, 'ckpt: ', ckpt, '-'*20)
            agent = MultiHeadDQNAgent(num_actions=num_actions, config=config)
            agent.load(kf.ckpt_path + 'offline_rem_{}.ckpt'.format(ckpt))

            agents = []
            for idx in range(kf.n_splits):
                config_idx = copy.deepcopy(config)
                config_idx['persistent_directory'] = kf.agent_paths[idx]
                config_idx['checkpoint_path'] = kf.ckpt_paths[idx]
                agent_idx = MultiHeadDQNAgent(num_actions=num_actions, config=config_idx)
                agent_idx.load(kf.ckpt_paths[idx] + 'offline_rem_{}.ckpt'.format(ckpt))
                agents.append(agent_idx)
            states, qvalues, qtildes = kf.update_q(agents, bc)

            print('-'*20, 'adv learner', '-'*20)
#             advs1 = qvalues - qvalues.mean(axis=1, keepdims=True)
#             agent1 = AdvantageLearner(num_actions=num_actions)
#             agent1._train(states, advs1)
            
            advs2 = qtildes - qtildes.mean(axis=1, keepdims=True)
            agent2 = AdvantageLearner(num_actions=num_actions)
            agent2._train(states, advs2)

            print('-'*20, 'fqe on dqn & seal', '-'*20)
            fqe_dqn = FQE(agent.greedy_actions, num_actions=num_actions, activation='tanh', hiddens=config['hiddens'], max_iter=100, eps=0.0015)
            fqe_dqn.train(trajs)
#             fqe_dml = FQE(agent1.greedy_actions, num_actions=num_actions)
#             fqe_dml.train(trajs)
            fqe_seal = FQE(agent2.greedy_actions, num_actions=num_actions, activation='tanh', hiddens=config['hiddens'], max_iter=100, eps=0.0015)
            fqe_seal.train(trajs)

            rets.loc[(fold, ckpt), 'dqn'] = fqe_dqn.values
#             rets.loc[(fold, ckpt), 'dml'] = fqe_dml.values
            rets.loc[(fold, ckpt), 'seal'] = fqe_seal.values
            
    return rets

In [2]:
save_path = './data/mh/rem/'
pool = mp.Pool(5)
rets = pool.map(one_step, range(5))
pool.close()

with open(save_path + 'rets_rem_mh.pkl', 'wb') as f:
    pickle.dump(rets, f)

-------------------- --------------------start -------------------- start -------------------- --------------------
start
 --------------------
----------------------------------------  startstart -------------------- --------------------

-------------------- training agent --------------------
-------------------- training agent --------------------
-------------------- training agent --------------------
-------------------- training agent ----------------------------------------
 training agent --------------------
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/209652396/fold0/train/agent/trajs.pkl!
Refresh buffer every 1000000 sampling!
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/218175338/fold0/train/agent/trajs.pkl!Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/932136058/fold0/train/agent/trajs.pkl!

Refresh buffer every 1000000 sampling!Refresh buffer every 1000000 sampling!

Loaded tra


saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold0/train/agent/ckpt/offline_rem_50000.ckpt
-------------------- training agents --------------------
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold0/train/agent0/trajs0.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X mo

Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/218175338/fold0/train/agent1/trajs1.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the l



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/218175338/fold0/train/agent2/ckpt/offline_rem_5000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold0/train/agent2/ckpt/offline_rem_10000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/932136058/fold0/train/agent2/ckpt/offline_rem_15000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/798842024/fold0/train/agent2/ckpt/offline_rem_15000.ckpt
saving model weights


saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/209652396/fold0/train/agent2/ckpt/offline_rem_40000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/218175338/fold0/train/agent2/ckpt/offline_rem_45000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/932136058/fold0/train/agent3/ckpt/offline_rem_5000.ckpt
saving model weights at /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold0/train/agent2/ckpt/offline_rem_50000.ckpt
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold0/train/agent3/trajs3.pkl!
Refresh buffer every 1000000 sampling!

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer

...


-----iteration:  0 target diff:  0.9196856783823851 values:  -57.92263 ----- 

-----iteration:  1 target diff:  0.0020236496047556117 values:  -57.921356 ----- 

-----iteration:  2 target diff:  0.0021210289614338584 values:  -57.86317 ----- 

-----iteration:  3 target diff:  0.001660571166152779 values:  -57.895428 ----- 

-----iteration:  4 target diff:  0.002221436751904834 values:  -57.871372 ----- 

-----iteration:  5 target diff:  0.0010958983995156741 values:  -57.945156 ----- 

-------------------- ckpt:  30000 --------------------
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold4/train/agent/trajs.pkl!
Refresh buffer every 1000000 sampling!
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold4/train/agent0/trajs0.pkl!
Refresh buffer every 1000000 sampling!
Loaded trajectories from load path: /home/jupyt/leyuan/SUPRL/data/mh/rem/tmp/717354021/fold4/train/agent1/trajs1.pkl!
Refresh buffer every 10



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

---------------

-----iteration:  1 target diff:  0.0025501036534178964 values:  -57.66332 ----- 

-----iteration:  2 target diff:  0.0018831396603604016 values:  -57.65239 ----- 

-----iteration:  3 target diff:  0.001682912753513148 values:  -57.67416 ----- 

-----iteration:  4 target diff:  0.0022668398145761688 values:  -57.708843 ----- 

-----iteration:  5 target diff:  0.0021345518500047065 values:  -57.664574 ----- 

-----iteration:  6 target diff:  0.0016894599587736821 values:  -57.671043 ----- 

-----iteration:  7 target diff:  0.0017416339519316067 values:  -57.748318 ----- 

-----iteration:  8 target diff:  0.0024643165447361475 values:  -57.774826 ----- 

-----iteration:  9 target diff:  0.0018990551717713172 values:  -57.708 ----- 

-----iteration:  10 target diff:  0.001639268971391225 values:  -57.73864 ----- 

-----iteration:  11 target diff:  0.0013949363925541058 values:  -57.722897 ----- 


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intende


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

-----iteration:  0 target diff:  0.9185546271277591 values:  -57.30308 ----- 

-----iteration:  1 target diff:  0.0032673995682701495 values:  -57.263077 ----- 

-----iteration:  2 target diff:  0.002402238392551516 values:  -57.274136 ----- 

-----iteration:  3 target diff:  0.0017338449493529337 values:  -57.235252 ----- 

-----iteration:  4 target diff:  0.0017427303728142353 values:  -57.20309 ----- 

-----iteration:  5 target diff:  0.0020569673183454926 values:  -57.087097 ----- 

-----iteration: 